#### This notebook will be used to create the dataframe for the time complexity analysis

In [15]:
from ortools.linear_solver import pywraplp
from numpy import average, std
import json
import pandas as pd
import requests
import math
import numpy as np
import time 
import warnings
from scipy.stats import wilcoxon
import plotly.express as px
warnings.filterwarnings('ignore')

In [16]:
def index2shift(i):
    return shifts[i]

def shift2index(shift):
    return shifts.index(shift)

def index2vehicle(j):
    return vehicles[j]

def vehicle2index(vehicle):
    return vehicles.index(vehicle)

def index2neighbourhood(k):
    return neighbourhoods[k]

def neighbourhood2index(neighbourhood):
    return neighbourhoods.index(neighbourhood)

def index2station(l):
    return stations[l]

def station2index(station):
    return stations.index(station)
def get_costs(l, vehicle, shift):
    return cost_matrix["normalized_cost" + vehicle + f"shift{shift}"][l]

In [17]:
total_vehicles = {
    "ha": np.float64(1),
    "engines": np.float64(197),
    "ladders": np.float64(143),
    "rescue": np.float64(5),
    "squads": np.float64(8),

}

In [18]:
response1 = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/distancias_estaciones_barrios.json")
distances_stations_ntas_db = json.loads(response1.text)
response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/estaciones.json")
stations_db = json.loads(response.text)
with open("./functions/neighbors_with_9_minute_station.json", "r") as f:
    neighbors_with_9_minute_station = json.load(f)
s = pd.read_csv("./aux_data/skl.csv",index_col=0)
d = pd.read_csv("./aux_data/dlk.csv",index_col=0)
b = pd.read_csv("./aux_data/bijk.csv",index_col=0)
Pkv = pd.read_csv("./aux_data/pkv.csv", index_col=0)
Skl =pd.read_csv("aux_data/skl.csv")

In [19]:
neighbourhoods = list(neighbors_with_9_minute_station.keys())
K = len(neighbourhoods)
shifts = ["9 AM to 6 PM", "6 PM until 9 AM"]
I = len(shifts)
stations = list(distances_stations_ntas_db.keys())
L = len(stations)
vehicles = ["engines", "squads", "ladders", "ha", "rescue"]
J = len(vehicles)

In [20]:
s = pd.read_csv("./aux_data/skl.csv",index_col=0)
d = pd.read_csv("./aux_data/dlk.csv",index_col=0)
b = pd.read_csv("./aux_data/bijk.csv",index_col=0)
Pkv = pd.read_csv("./aux_data/pkv.csv", index_col=0)
Skl =pd.read_csv("aux_data/skl.csv")

In [21]:
s.index = pd.Series(list(s.index)).apply(lambda x: station2index(x))
s.columns = pd.Series(list(s.columns)).apply(lambda x: neighbourhood2index(x))
d.index = pd.Series(list(d.index)).apply(lambda x: station2index(x))
d.columns = pd.Series(list(d.columns)).apply(lambda x: neighbourhood2index(x))

In [22]:
Cl = {}
for st in stations_db:
    Cl[stations.index(st["address"])] = st["capacity"]

In [23]:
first = b[b['shift']==1]
second = b[b['shift']==0]
aux = first['nta']
for elem in neighbourhoods:
    if elem not in aux:
        df = pd.Series(data={'nta':elem,'engines':1,'squads':1,'ladders':1,'ha':1,'rescue':1,'shift':1})
        first.loc[len(first)]=df

first = first.drop_duplicates(subset=['nta']).reset_index().drop(['index'],axis=1)
aux1 = second['nta']
for elem in neighbourhoods:
    if elem not in aux1:
        df = pd.Series(data={'nta':elem,'engines':1,'squads':1,'ladders':1,'ha':1,'rescue':1,'shift':0})
        second.loc[len(second)]=df

second = second.drop_duplicates(subset=['nta']).reset_index().drop(['index'],axis=1)

aux1 = second['nta']
for elem in neighbourhoods:
    if elem not in aux1:
        df = pd.Series(data={'nta':elem,'engines':1,'squads':1,'ladders':1,'ha':1,'rescue':1,'shift':0})
        second.loc[len(second)]=df


second = second.drop_duplicates(subset=['nta']).reset_index().drop(['index'],axis=1)

second = second.drop_duplicates(subset=['nta']).reset_index().drop(['index'],axis=1)
b= pd.concat([first,second],axis=0).reset_index().drop(["index"],axis=1)

In [24]:
cost_matrix = d*s
cost = {}
for i in range(len(b)):
      cost[(neighbourhood2index(b.iloc[i]["nta"]), "ha", b.iloc[i]["shift"])] = b.iloc[i]["ha"]
      cost[(neighbourhood2index(b.iloc[i]["nta"]), "squads", b.iloc[i]["shift"])] = b.iloc[i]["squads"]
      cost[(neighbourhood2index(b.iloc[i]["nta"]), "rescue", b.iloc[i]["shift"])] = b.iloc[i]["rescue"]

costs = {}
for i in range(len(cost_matrix)):
      costs[("ha", 0)] = costs.get(("ha", 0), []) + [sum(cost_matrix.iloc[i][j]*(cost.get((j, "ha", 0), 0)) for j in range(cost_matrix.shape[1]))]
      costs[("ha", 1)] = costs.get(("ha", 1), []) + [sum(cost_matrix.iloc[i][j]*(cost.get((j, "ha", 1), 0)) for j in range(cost_matrix.shape[1]))]
      costs[("squads", 0)] = costs.get(("squads", 0), []) + [sum(cost_matrix.iloc[i][j]*(cost.get((j, "squads", 0), 0)) for j in range(cost_matrix.shape[1]))]
      costs[("squads", 1)] = costs.get(("squads", 1), []) + [sum(cost_matrix.iloc[i][j]*(cost.get((j, "squads", 1), 0)) for j in range(cost_matrix.shape[1]))]
      costs[("rescue", 0)] = costs.get(("rescue", 0), []) + [sum(cost_matrix.iloc[i][j]*(cost.get((j, "rescue", 0), 0)) for j in range(cost_matrix.shape[1]))]
      costs[("rescue", 1)] = costs.get(("rescue", 1), []) + [sum(cost_matrix.iloc[i][j]*(cost.get((j, "rescue", 1), 0)) for j in range(cost_matrix.shape[1]))]

cost_matrix["costhashift0"] = costs[("ha", 0)]
cost_matrix["costhashift1"] = costs[("ha", 1)]
cost_matrix["costsquadsshift0"] = costs[("squads", 0)]
cost_matrix["costsquadsshift1"] = costs[("squads", 1)]
cost_matrix["costrescueshift0"] = costs[("rescue", 0)]
cost_matrix["costrescueshift1"] = costs[("rescue", 1)]
cost_matrix["reacheable_neighbourhoods"] = s.apply(lambda x:sum(x),axis=1)  #total reacheable centers
cost_matrix["normalized_costhashift0"] = cost_matrix.apply(lambda x: x["costhashift0"]/x["reacheable_neighbourhoods"],axis=1)
cost_matrix["normalized_costhashift1"] = cost_matrix.apply(lambda x: x["costhashift1"]/x["reacheable_neighbourhoods"],axis=1)
cost_matrix["normalized_costsquadsshift0"] = cost_matrix.apply(lambda x: x["costsquadsshift0"]/x["reacheable_neighbourhoods"],axis=1)
cost_matrix["normalized_costsquadsshift1"] = cost_matrix.apply(lambda x: x["costsquadsshift1"]/x["reacheable_neighbourhoods"],axis=1)
cost_matrix["normalized_costrescueshift0"] = cost_matrix.apply(lambda x: x["costrescueshift0"]/x["reacheable_neighbourhoods"],axis=1)
cost_matrix["normalized_costrescueshift1"] = cost_matrix.apply(lambda x: x["costrescueshift1"]/x["reacheable_neighbourhoods"],axis=1)
cost_matrix["normalized_costhashift0"] = cost_matrix["normalized_costhashift0"].apply(lambda x: (x-min(cost_matrix["normalized_costhashift0"]))/(max(cost_matrix["normalized_costhashift0"])- min(cost_matrix["normalized_costhashift0"])))
cost_matrix["normalized_costhashift1"] = cost_matrix["normalized_costhashift1"].apply(lambda x: (x-min(cost_matrix["normalized_costhashift1"]))/(max(cost_matrix["normalized_costhashift1"])- min(cost_matrix["normalized_costhashift1"])))
cost_matrix["normalized_costsquadsshift0"] = cost_matrix["normalized_costsquadsshift0"].apply(lambda x: (x-min(cost_matrix["normalized_costsquadsshift0"]))/(max(cost_matrix["normalized_costsquadsshift0"])- min(cost_matrix["normalized_costsquadsshift0"])))
cost_matrix["normalized_costsquadsshift1"] = cost_matrix["normalized_costsquadsshift1"].apply(lambda x: (x-min(cost_matrix["normalized_costsquadsshift1"]))/(max(cost_matrix["normalized_costsquadsshift1"])- min(cost_matrix["normalized_costsquadsshift1"])))
cost_matrix["normalized_costrescueshift0"] = cost_matrix["normalized_costrescueshift0"].apply(lambda x: (x-min(cost_matrix["normalized_costrescueshift0"]))/(max(cost_matrix["normalized_costrescueshift0"])- min(cost_matrix["normalized_costrescueshift0"])))
cost_matrix["normalized_costrescueshift1"] = cost_matrix["normalized_costrescueshift1"].apply(lambda x: (x-min(cost_matrix["normalized_costrescueshift1"]))/(max(cost_matrix["normalized_costrescueshift1"])- min(cost_matrix["normalized_costrescueshift1"])))
cost_matrix=cost_matrix.fillna(2**8)

In [25]:
factor_dict = {}
for v in ["ha","squads","rescue"]:
    for i in [0,1]:
        for n in list(b["nta"].unique()):
            factor_dict[(i,neighbourhood2index(n),v)] = b[v].loc[(b["nta"]==n)&(b["shift"]==i)].values

In [26]:
Bijk = b
newcols = []
for col in Bijk.columns[1:-1]:
    newcols.append(vehicle2index(col))
Bijk.columns = ["nta"] + newcols + ["shift"]
Bijk["nta"] = Bijk["nta"].apply(lambda x: neighbourhood2index(x))
Bijk = Bijk.apply(lambda x: x.apply(lambda y: math.ceil(y)))

In [27]:
newcols = []
for col in Skl.columns[1:]:
    newcols.append(neighbourhood2index(col))
Skl.columns = ["station"]+newcols
Skl["station"] = Skl["station"].apply(lambda x: station2index(x))


In [28]:
new_param=[]
cols = cost_matrix.columns[0:195]
for c in cols:
    suma = 0
    for e in cost_matrix[c]:
        suma+=e

    if suma >0:new_param.append(1)
    else:new_param.append(0)

In [29]:
boroughs = list(Pkv["index"].unique())

def index2boro(l):
    return boroughs[l]

def boro2index(station):
    return boroughs.index(station)

Pkv["index"] = Pkv["index"].apply(lambda x: boro2index(x))
Pkv.columns = np.hstack((np.array("Borough"),pd.Series(list(Pkv.columns[1:])).apply(lambda x: neighbourhood2index(x)).values))
B = len(Pkv["Borough"].unique())


In [30]:
solver_params = {}

solver_types = ["SCIP_MIXED_INTEGER_PROGRAMMING","CBC"]
for algorithm in solver_types:
  solver_params[algorithm] = {"objective_function":[],"time":[]}
  for i in range(30):
    solver = pywraplp.Solver.CreateSolver(algorithm)
    variables = {}
    for i in range(I):
      for j in range(J):
        if j not in {1, 3, 4}: # If the vehicle is not of squad, hazardous or rescue type
          for k in range(K):
            for l in range(L):
              # Shifted to integer variable instead of real
              v = solver.IntVar(0, solver.infinity(), "Vehicles of type " + index2vehicle(j) + " assigned to station " + index2station(l) + " to cover the neighbourhood " + index2neighbourhood(k) + " in shift " + index2shift(i))
              variables[(i,j,k,l)] = v
        else:
          for l in range(L):
            v = solver.BoolVar("Vehicles of type " + index2vehicle(j) + " assigned to station " + index2station(l) + " in shift " + index2shift(i))
            variables[("Y",i,j,l)] = v

    for l1 in range(L):
      for l2 in range(L):
        v = solver.IntVar(0, solver.infinity(), "Vehicles moved from station station " + index2station(l1) + " to station " + index2station(l2) + " in the change of shift (day->night)")
        variables[("N",0,l1,l2)] = v
        v = solver.IntVar(0, solver.infinity(), "Vehicles moved from station station " + index2station(l1) + " to station " + index2station(l2) + " in the change of shift (night->day)")
        variables[("N",1,l1,l2)] = v

    for i in range(I):
      for l in range(L):
        v = solver.BoolVar("There is more than one vehicle in the station " + index2station(l) + " during shift " + index2shift(i))
        variables[(i,l)] = v
        
    max_sc = Bijk[[0,2]].sum().sum()
    combinations = {}
    beta = 0
    objective = solver.Objective()
    objective.SetMinimization()
    for i in range(I):
      for j in range(J):
        if j not in (1,3,4): #Hazmat,squat and rescue units are not included in this summation
          for k in range(K):
            try:
              # The neighbourhood k has needs of vehicles of type j in shift i
              beta += int(Bijk[(Bijk["nta"] == k) & (Bijk["shift"] == i)][j].item())
              combinations[i,j,k] = "Available"
              for l in range(L):
                v = variables[(i,j,k,l)]
                objective.SetCoefficient(v/max_sc,-1)
            except:
              combinations[i,j,k] = "Not available"
    objective.SetOffset(beta/max_sc)

    #Squad vehicle
    for i in range(I):
        for l in range(L):
            variable = variables[("Y",i,1,l)]
            objective.SetCoefficient(variable,float(get_costs(l, "squads", i)/np.float64(total_vehicles["squads"]))) #Precoumputed costs are used
    
    #Hazardous-Hazmat vehicle
    for i in range(I):
        for l in range(L):
            variable = variables[("Y",i,3,l)]
            objective.SetCoefficient(variable,float(get_costs(l, "ha", i)/np.float64(total_vehicles["ha"])))
          
    #Rescue vehicle
    for i in range(I):
        for l in range(L):
            variable = variables[("Y",i,4,l)]
            objective.SetCoefficient(variable,float(get_costs(l, "rescue", i)/np.float64(total_vehicles["rescue"])))      
            
    for l1 in range(L):
      for l2 in range(L):
        for i in range(I):
          variable = variables[("N",i,l1,l2)]
          objective.SetCoefficient(variable,1/max_sc)

    limit = []

    for i in range(I):
      for j in range(J):
        for k in range(K):
            bijk = int(Bijk[(Bijk["nta"] == k) & (Bijk["shift"] == i)][j])
            c = solver.Constraint(0, bijk)
            for l in range(L):
              if (i,j,k,l) in variables:
                v = variables[(i,j,k,l)]
                c.SetCoefficient(v, 1)
            limit.append(c)
            
    haz = []

    for i in range(I):
      c = solver.Constraint(1, 1)
      for l in range(L):
        v = variables[("Y",i,3,l)]
        c.SetCoefficient(v, 1)
      haz.append(c)
      
    squads = []

    for i in range(I):
      c = solver.Constraint(8, 8)
      for l in range(L):
        v = variables[("Y",i,1,l)]
        c.SetCoefficient(v, 1)
      squads.append(c)
      
    rescue = []

    for i in range(I):
      c = solver.Constraint(5, 5)
      for l in range(L):
        v = variables[("Y",i,4,l)]
        c.SetCoefficient(v, 1)
      rescue.append(c)
      
    vehiclej = []

    for j in range(J):
      if j not in {1,3,4}:
        for i in range(I):
          c = solver.Constraint(0, total_vehicles[index2vehicle(j)])
          for l in range(L):
            for k in range(K):
              v = variables[(i,j,k,l)]
              c.SetCoefficient(v, 1)
          vehiclej.append(c)
          
    nine_minutes = []

    for k in range(K):
      p = new_param[k]
      for i in range(I):
        c = solver.Constraint(p, solver.infinity())
        for l in range(L):
          skl = s.iloc[l,k]
          for j in range(J):
            if j not in {1, 3, 4}:
              v = variables[(i,j,k,l)]
              c.SetCoefficient(v, skl)
            else:
              v = variables[("Y",i,j,l)]
              c.SetCoefficient(v, skl)

        nine_minutes.append(c)
        
    capacity = []

    for l in range(L):
      cl = Cl[l]
      for i in range(I):
        c = solver.Constraint(-solver.infinity(), cl)
        for j in range(J):
          if j not in {1,3,4}:
            for k in range(K):
              v = variables[(i,j,k,l)]
              c.SetCoefficient(v, 1)
          else:
            v = variables[("Y",i,j,l)]
            c.SetCoefficient(v, 1)
          capacity.append(c)
          
    displacement2night = []

    for l in range(L):
        c = solver.Constraint(-solver.infinity(), 0)
        v = variables[("Y",0,1,l)]
        c.SetCoefficient(v, 1)
        v = variables[("Y",1,1,l)]
        c.SetCoefficient(v, -1)
        v = variables[("Y",1,3,l)]
        c.SetCoefficient(v, -1)
        v = variables[("Y",0,3,l)]
        c.SetCoefficient(v, 1)
        v = variables[("Y",1,4,l)]
        c.SetCoefficient(v, -1)
        v = variables[("Y",0,4,l)]
        c.SetCoefficient(v, 1)
        for l2 in range(L):
            v = variables[("N",0,l,l2)]
            c.SetCoefficient(v, -1)
            v = variables[("N",0,l2,l)]
            c.SetCoefficient(v, 1)
        for j in range(J):
          if j not in {1,3,4}:
            for k in range(K):
              v = variables[(0,j,k,l)]
              c.SetCoefficient(v, 1)
              v = variables[(1,j,k,l)]
              c.SetCoefficient(v, -1)
        displacement2night.append(c)
        
    displacement2day = []

    for l in range(L):
        c = solver.Constraint(-solver.infinity(), 0)
        v = variables[("Y",0,1,l)]
        c.SetCoefficient(v, -1)
        v = variables[("Y",1,1,l)]
        c.SetCoefficient(v, 1)
        v = variables[("Y",1,3,l)]
        c.SetCoefficient(v, 1)
        v = variables[("Y",0,3,l)]
        c.SetCoefficient(v, -1)
        v = variables[("Y",1,4,l)]
        c.SetCoefficient(v, 1)
        v = variables[("Y",0,4,l)]
        c.SetCoefficient(v, -1)
        for l2 in range(L):
            v = variables[("N",1,l,l2)]
            c.SetCoefficient(v, -1)
            v = variables[("N",1,l2,l)]
            c.SetCoefficient(v, 1)
        for j in range(J):
          if i not in {1,3,4}:
            for k in range(K):
              v = variables[(0,j,k,l)]
              c.SetCoefficient(v, -1)
              v = variables[(1,j,k,l)]
              c.SetCoefficient(v, 1)
        displacement2day.append(c)
        
    fair_distribution = []

    for bo in range(B):
        c = solver.Constraint(-solver.infinity(), 0)
        for k in range(k):
          p = int(Pkv.iloc[bo, k+1])
          for j in range(J):
            if j not in {1,3,4}:
              for l in range(L):
                for i in range(I):
                  v = variables[(i,j,k,l)]
                  c.SetCoefficient(v, p)
                  c.SetCoefficient(v, -0.3)
        fair_distribution.append(c)

    fair_distribution2 = []

    for bo in range(B):
        for i in range(I):
          for j in (1,4):
            c = solver.Constraint(1, solver.infinity())
            for k in range(K):
              p = float(Pkv.iloc[bo, k+1])
              if p:
                for l in range(L):
                    skl = int(s.iloc[l,k])
                    v = variables[("Y",i,j,l)]
                    c.SetCoefficient(v, skl)
            fair_distribution2.append(c)
        
    more_than_one = []

    for i in range(I):
      for l in range(L):
        c = solver.Constraint(0, solver.infinity())
        v = variables[(i,l)]
        c.SetCoefficient(v, -2)
        if j not in {1,3,4}:
            for k in range(K):
              v = variables[(i,j,k,l)]
              c.SetCoefficient(v, 1)
        else:
            v = variables[("Y",i,j,l)]
            c.SetCoefficient(v, 1)
        more_than_one.append(c)

    for i in range(I):
      cl = Cl[l]
      for i in range(I):
        c = solver.Constraint(-solver.infinity(), 1)
        v = variables[(i,l)]
        c.SetCoefficient(v, -cl)
        if j not in {1,3,4}:
            for k in range(K):
              v = variables[(i,j,k,l)]
              c.SetCoefficient(v, 1)
        else:
            v = variables[("Y",i,j,l)]
            c.SetCoefficient(v, 1)
        more_than_one.append(c)
        
    not_just_one_type = []

    for i in range(I):
        for j in range(J):
          for l in range(L):
            c = solver.Constraint(-solver.infinity(), 0)
            v = variables[(i,l)]
            c.SetCoefficient(v, -1)
            for k in range(k):
                v = variables[(i,j,k,l)]
                c.SetCoefficient(v, 1)
                for j2 in range(J):
                    if j2 not in {1,3,4}:
                      v = variables[(i,j2,k,l)]
                      c.SetCoefficient(v, -1)
                    else:
                      v = variables[("Y",i,j2,l)]
                      c.SetCoefficient(v, -1)
        not_just_one_type.append(c)
    
    
    t = time.time()
    solver.Solve()
    solver_params[algorithm]["time"].append(time.time()-t)
    solver_params[algorithm]["objective_function"].append(objective.Value())
    print(solver_params)

{'SCIP_MIXED_INTEGER_PROGRAMMING': {'objective_function': [2.447299242612048], 'time': [33.110724449157715]}}
{'SCIP_MIXED_INTEGER_PROGRAMMING': {'objective_function': [2.447299242612048, 2.447299242612048], 'time': [33.110724449157715, 31.54045557975769]}}
{'SCIP_MIXED_INTEGER_PROGRAMMING': {'objective_function': [2.447299242612048, 2.447299242612048, 2.447299242612048], 'time': [33.110724449157715, 31.54045557975769, 34.32983040809631]}}
{'SCIP_MIXED_INTEGER_PROGRAMMING': {'objective_function': [2.447299242612048, 2.447299242612048, 2.447299242612048, 2.447299242612048], 'time': [33.110724449157715, 31.54045557975769, 34.32983040809631, 32.984302043914795]}}
{'SCIP_MIXED_INTEGER_PROGRAMMING': {'objective_function': [2.447299242612048, 2.447299242612048, 2.447299242612048, 2.447299242612048, 2.447299242612048], 'time': [33.110724449157715, 31.54045557975769, 34.32983040809631, 32.984302043914795, 34.46614146232605]}}
{'SCIP_MIXED_INTEGER_PROGRAMMING': {'objective_function': [2.4472992

In [31]:
data_time = pd.DataFrame({
    "Time CBC": solver_params["CBC"]["time"] + solver_params["SCIP_MIXED_INTEGER_PROGRAMMING"]["time"],
    "Solver": ["CBC"] * len(solver_params["CBC"]["time"]) + ["SCIP"] * len(solver_params["CBC"]["time"]),
    "Iteration": [i for i in range(len(solver_params["CBC"]["time"]))] * 2  
})

In [32]:
#data_time.to_csv("aux_data/tiempos.csv")

In [33]:
pd.DataFrame({"Solver":["CBC","SCIP"],"Average Time":[np.mean(solver_params["CBC"]["time"]),np.mean(solver_params["SCIP_MIXED_INTEGER_PROGRAMMING"]["time"])],"Std. Time":[np.std(solver_params["CBC"]["time"]),np.std(solver_params["SCIP_MIXED_INTEGER_PROGRAMMING"]["time"])]})

,Solver,Average Time,Std. Time
0,CBC,5.428266,0.425659
1,SCIP,31.314409,4.271904


In [34]:
wilcoxon(solver_params["CBC"]["time"],solver_params["SCIP_MIXED_INTEGER_PROGRAMMING"]["time"])

WilcoxonResult(statistic=0.0, pvalue=1.7343976283205784e-06)

##### As the p-value reflects, taking a level of significance of 95%, we can confirm that there is a statististically relevant  difference between the mean time of execution of the two solvers considered